# closure phases

[L. Blackburn, Sep 2018; rewritten for Python 3.9, Dec 2022]

The purpose of this test is to check trivial and non-trivial closure phases for sanity.

In [ ]:
# basic import and helper functions
import pandas as pd
from eat.io import hops, util
from eat.hops import util as hu
from eat.plots import util as pu
import matplotlib.pyplot as plt
import os
import sys
import numpy as np
import seaborn as sns

sns.reset_orig()
# sns.set_palette(sns.color_palette(sns.hls_palette(16, l=.6, s=.6)))
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# %config InlineBackend.figure_formats=['svg']

nb_stdout = sys.stdout # grab for later

def wide(w=8, h=3): plt.setp(plt.gcf(), figwidth=w, figheight=h); \
    plt.tight_layout()

def tightx(): plt.autoscale(enable=True, axis='x', tight=True)

def multline(xs, fun=plt.axvline):
    for x in xs: fun(x, alpha=0.25, ls='--', color='k')

def toiter(x):
    return(x if hasattr(x, '__iter__') else [x,])

# pd.options.display.float_format = '{:,.6f}'.format
from IPython.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))

Loading the alist file (for computing global scan numbers) and the scan-averaged closure files for LL and RR polarizations for plotting closure quantities.

In [ ]:
# Define variables and load data
alistf = 'alist.v6'
alistfll = 'alist.v6.8s.LL.close.avg'
alistfrr = 'alist.v6.8s.RR.close.avg'
datadir = os.environ['DATADIR']

a = hops.read_alist(os.path.join(datadir, alistf)) # alist file to make scan_no

ll = hops.read_tlist_v6(os.path.join(datadir, alistfll))
ll['polarization'] = 'LL'

rr = hops.read_tlist_v6(os.path.join(datadir, alistfrr))
rr['polarization'] = 'RR'

# Concat the two dataframes and pre-process them
df_close = pd.concat((ll, rr), ignore_index=True)
util.add_gmst(df_close)
hu.setparity(df_close)
util.fix(df_close)

In [ ]:
# Add scan_no to the concatenated dataframe for grouping and plotting
util.add_scanno(a)
tup2scanno = a.groupby(['expt_no', 'scan_id']).first().scan_no
df_close = df_close.join(tup2scanno, on=['expt_no', 'scan_id'], how='left')

In [ ]:
# Compute the boundaries between expt_nos
sorted_a = a.sort_values(['expt_no', 'scan_no'])
last_scans = sorted_a.groupby('expt_no')['scan_no'].max() # Find the 'max' scan_no for each expt_no
elines = (last_scans.iloc[:-1] + 0.5).to_numpy() # Drop the final expt_no and offset by 0.5

### Plot trivial closure phases

Generate plots of closure phase (bispectrum phase) vs scan number for all triangles containing the "zero-baselines" ALMA-APEX and SMA-JCMT.

In [ ]:
def clplot(df_close, triangles, threshold=3):
    df = df_close[(df_close.bis_snr > threshold) & ~df_close.triangle.str.contains('R') & (df_close.duration > 8*5)]
    for tri in triangles:
        rr = df[(df.polarization == 'RR') & (df.triangle==tri)]
        ll = df[(df.polarization == 'LL') & (df.triangle==tri)]
        hl = plt.errorbar(ll.scan_no - 0.025, ll.bis_phas, yerr=(1./ll.bis_snr)*180./np.pi,
                          fmt='o', label=tri)
        _ = plt.errorbar(rr.scan_no + 0.025, rr.bis_phas, yerr=(1./rr.bis_snr)*180./np.pi,
                          fmt='x', label='_nolegend_', color=hl[0].get_color())
    
    plt.gca().yaxis.grid(alpha=0.25)
    plt.legend(ncol=3)

    plt.title('closure phases')
    plt.xlabel('scan number')
    plt.ylabel('degrees')

    pu.multline(elines)

In [ ]:
# Plot closure phases for triangles with A and X
clplot(df_close, sorted((t for t in set(df_close.triangle) if 'A' in t and 'X' in t and 'R' not in t)))
wide(12, 5)

In [ ]:
# Plot closure phases for triangles with S and J
clplot(df_close, sorted((t for t in set(df_close.triangle) if 'S' in t and 'J' in t and 'R' not in t)))
wide(12, 5)

### Plot other closure phases by source and track (expt_no)

Here we plot closure phases vs time (GMST) for all remaining triangles by source.

In [ ]:
def clplot2(df_close, source, triangles, threshold=3):
    df = df_close[(df_close.bis_snr > threshold) & ~df_close.triangle.str.contains('R') & (df_close.duration > 8*5) 
           & (df_close.source==source) & df_close.triangle.isin(set(triangles))].copy()
    
    # Adjust GMST to be continuous across the entire observing track
    t = np.hstack((df.gmst.sort_values().values, df.gmst.sort_values().values + 24.))
    if len(t) == 0:
        return True
    idx = np.argmax(np.diff(t))
    toff = np.fmod(48. - 0.5 * (t[idx] + t[1+idx]), 24.)
    df.gmst = np.fmod(df.gmst + toff, 24.) - toff
    
    for tri in triangles:
        rr = df[(df.polarization == 'RR') & (df.triangle==tri)]
        ll = df[(df.polarization == 'LL') & (df.triangle==tri)]
        if(len(ll) > 0):
            (llabel, rlabel) = (tri, '_nolegend_')
        elif(len(rr) > 0):
            (llabel, rlabel) = ('_nolegend_', tri)
        else:
            (llabel, rlabel) = ('_nolegend_', '_nolegend_')
        hl = plt.errorbar(ll.gmst - 0.01, ll.bis_phas, yerr=(1./ll.bis_snr)*180./np.pi,
                          fmt='o', label=llabel)
        _ = plt.errorbar(rr.gmst + 0.01, rr.bis_phas, yerr=(1./rr.bis_snr)*180./np.pi,
                          fmt='x', label=rlabel, color=hl[0].get_color())
        
    plt.gca().yaxis.grid(alpha=0.25)
    plt.legend(ncol=3)
    
    plt.title(source + ' closure phases')
    plt.xlabel('GMST hour')
    plt.ylabel('degrees')

Plot closure phases against time for all non-degenerate triangles by source.

In [ ]:
triangles = sorted((t for t in set(df_close.triangle) if 'R' not in t))

# Filter out triangles that contain 'X' or 'J'; use only 'A' or 'S' instead for baselines to Chile and Hawaii
triangles = [t for t in triangles if not ('X' in t or 'J' in t)]

sns.set_palette(sns.color_palette(sns.hls_palette(len(triangles), l=.6, s=.6)))

for src in sorted(set(df_close.source)):
    ret = clplot2(df_close, source=src, triangles=triangles)
    if ret:
        continue
    plt.ylim(-180, 180)
    wide(12, 5)
    plt.show()

Plot closure phases against time for all non-degenerate triangles by source, further subdivided by track (expt no).

In [ ]:
triangles = sorted((t for t in set(df_close.triangle) if 'R' not in t))

# Filter out triangles that contain 'X' or 'J'; use only 'A' or 'S' instead for baselines to Chile and Hawaii
triangles = [t for t in triangles if not ('X' in t or 'J' in t)]

sources = ['SGRA', 'M87', '3C279', 'OJ287']
for src in sorted(set(sources)):
    for expt_no in sorted(set(df_close.expt_no)):
        ret = clplot2(df_close[df_close.expt_no == expt_no], source=src, triangles=triangles)
        if ret:
            continue
        plt.ylim(-180, 180)
        plt.title(str(expt_no) + ' - ' + src + ' closure phases')
        wide(12, 5)
        plt.show()